## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [1]:
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

albums
['AlbumId', 'Title', 'ArtistId']
[('AlbumId', Column('AlbumId', INTEGER(), table=<albums>, primary_key=True, nullable=False)), ('Title', Column('Title', NVARCHAR(length=160), table=<albums>, nullable=False)), ('ArtistId', Column('ArtistId', INTEGER(), ForeignKey('artists.ArtistId'), table=<albums>, nullable=False))]
-------------------------
artists
['ArtistId', 'Name']
[('ArtistId', Column('ArtistId', INTEGER(), table=<artists>, primary_key=True, nullable=False)), ('Name', Column('Name', NVARCHAR(length=120), table=<artists>))]
-------------------------
customers
['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'SupportRepId']
[('CustomerId', Column('CustomerId', INTEGER(), table=<customers>, primary_key=True, nullable=False)), ('FirstName', Column('FirstName', NVARCHAR(length=40), table=<customers>, nullable=False)), ('LastName', Column('LastName', NVARCHAR(length=20), table=<customers>, nullable=F

In [3]:
from sqlalchemy import select
#  A seguir um exemplo de query na tabela Albums
# usamos o objeto Base para acessar cada modelo ORM.

session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(first_album.AlbumId, first_album.Title)

1 For Those About To Rock We Salute You


#### Q3. 
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [4]:
from sqlalchemy.orm import Session
from sqlalchemy import select

# Criar uma sessão
with Session(engine) as session:
    # Consultar os três primeiros registros da tabela tracks
    Tracks = Base.classes.tracks  # Obter a classe mapeada para a tabela tracks
    res = session.execute(select(Tracks).limit(3))  # Limitar a consulta aos 3 primeiros registros
    tracks = res.scalars().all()  # Extrair os objetos ORM da consulta

    # Imprimir os três primeiros registros
    for track in tracks:
        print(f"TrackId: {track.TrackId}, Name: {track.Name}, AlbumId: {track.AlbumId}, GenreId: {track.GenreId}")

TrackId: 1, Name: For Those About To Rock (We Salute You), AlbumId: 1, GenreId: 1
TrackId: 2, Name: Balls to the Wall, AlbumId: 2, GenreId: 1
TrackId: 3, Name: Fast As a Shark, AlbumId: 3, GenreId: 1


In [5]:
from sqlalchemy.orm import Session
from sqlalchemy import select

# Obter as classes mapeadas para as tabelas tracks e albums
Tracks = Base.classes.tracks
Albums = Base.classes.albums

# Criar uma sessão
with Session(engine) as session:
    # Fazer uma consulta para obter o nome da faixa e o título do álbum das primeiras 20 faixas
    query = (
        select(Tracks.Name, Albums.Title)
        .join(Albums, Tracks.AlbumId == Albums.AlbumId)
        .limit(20)  # Limitar o resultado às primeiras 20 faixas
    )
    result = session.execute(query).all()

    # Imprimir o nome da faixa e o título do álbum
    for track_name, album_title in result:
        print(f"Track Name: {track_name}, Album Title: {album_title}")

Track Name: For Those About To Rock (We Salute You), Album Title: For Those About To Rock We Salute You
Track Name: Put The Finger On You, Album Title: For Those About To Rock We Salute You
Track Name: Let's Get It Up, Album Title: For Those About To Rock We Salute You
Track Name: Inject The Venom, Album Title: For Those About To Rock We Salute You
Track Name: Snowballed, Album Title: For Those About To Rock We Salute You
Track Name: Evil Walks, Album Title: For Those About To Rock We Salute You
Track Name: C.O.D., Album Title: For Those About To Rock We Salute You
Track Name: Breaking The Rules, Album Title: For Those About To Rock We Salute You
Track Name: Night Of The Long Knives, Album Title: For Those About To Rock We Salute You
Track Name: Spellbound, Album Title: For Those About To Rock We Salute You
Track Name: Balls to the Wall, Album Title: Balls to the Wall
Track Name: Fast As a Shark, Album Title: Restless and Wild
Track Name: Restless and Wild, Album Title: Restless and Wi

In [ ]:
from sqlalchemy.orm import Session
from sqlalchemy import select

# Obter as classes mapeadas para as tabelas invoice_items e tracks
InvoiceItems = Base.classes.invoice_items
Tracks = Base.classes.tracks

# Criar uma sessão
with Session(engine) as session:
    # Fazer uma consulta para obter as 10 primeiras vendas
    query = (
        select(InvoiceItems.Quantity, Tracks.Name)
        .join(Tracks, InvoiceItems.TrackId == Tracks.TrackId)
        .limit(10)  # Limitar o resultado às 10 primeiras vendas
    )
    result = session.execute(query).all()

    # Imprimir os nomes das faixas vendidas e a quantidade vendida
    for quantity, track_name in result:
        print(f"Track Name: {track_name}, Quantity Sold: {quantity}")

Track Name: Balls to the Wall, Quantity Sold: 1
Track Name: Restless and Wild, Quantity Sold: 1
Track Name: Put The Finger On You, Quantity Sold: 1
Track Name: Inject The Venom, Quantity Sold: 1
Track Name: Evil Walks, Quantity Sold: 1
Track Name: Breaking The Rules, Quantity Sold: 1
Track Name: Dog Eat Dog, Quantity Sold: 1
Track Name: Overdose, Quantity Sold: 1
Track Name: Love In An Elevator, Quantity Sold: 1
Track Name: Janie's Got A Gun, Quantity Sold: 1


In [7]:
from sqlalchemy.orm import Session
from sqlalchemy import select, func, desc

# Obter as classes mapeadas para as tabelas invoice_items e tracks
InvoiceItems = Base.classes.invoice_items
Tracks = Base.classes.tracks

# Criar uma sessão
with Session(engine) as session:
    # Consultar as 10 faixas mais vendidas
    query = (
        select(Tracks.Name, func.sum(InvoiceItems.Quantity).label("total_sold"))
        .join(Tracks, InvoiceItems.TrackId == Tracks.TrackId)
        .group_by(Tracks.Name)
        .order_by(desc("total_sold"))  # Ordenar pela quantidade vendida em ordem decrescente
        .limit(10)  # Limitar aos 10 primeiros resultados
    )
    result = session.execute(query).all()

    # Imprimir os nomes das faixas mais vendidas e o número de vezes que foram vendidas
    for track_name, total_sold in result:
        print(f"Track Name: {track_name}, Total Sold: {total_sold}")

Track Name: The Trooper, Total Sold: 5
Track Name: Untitled, Total Sold: 4
Track Name: The Number Of The Beast, Total Sold: 4
Track Name: Sure Know Something, Total Sold: 4
Track Name: Hallowed Be Thy Name, Total Sold: 4
Track Name: Eruption, Total Sold: 4
Track Name: Where Eagles Dare, Total Sold: 3
Track Name: Welcome Home (Sanitarium), Total Sold: 3
Track Name: Sweetest Thing, Total Sold: 3
Track Name: Surrender, Total Sold: 3


In [8]:
from sqlalchemy.orm import Session
from sqlalchemy import select, func, desc

# Obter as classes mapeadas para as tabelas invoice_items, tracks, albums e artists
InvoiceItems = Base.classes.invoice_items
Tracks = Base.classes.tracks
Albums = Base.classes.albums
Artists = Base.classes.artists

# Criar uma sessão
with Session(engine) as session:
    # Consultar os 10 artistas que mais venderam
    query = (
        select(Artists.Name, func.sum(InvoiceItems.Quantity).label("total_sold"))
        .join(Tracks, InvoiceItems.TrackId == Tracks.TrackId)        # Join com tracks
        .join(Albums, Tracks.AlbumId == Albums.AlbumId)             # Join com albums
        .join(Artists, Albums.ArtistId == Artists.ArtistId)         # Join com artists
        .group_by(Artists.Name)                                     # Agrupar por artista
        .order_by(desc("total_sold"))                               # Ordenar pela soma decrescente
        .limit(10)                                                  # Limitar aos 10 primeiros
    )
    result = session.execute(query).all()

    # Imprimir os nomes dos artistas e o número total de vendas
    for artist_name, total_sold in result:
        print(f"Artist Name: {artist_name}, Total Sold: {total_sold}")

Artist Name: Iron Maiden, Total Sold: 140
Artist Name: U2, Total Sold: 107
Artist Name: Metallica, Total Sold: 91
Artist Name: Led Zeppelin, Total Sold: 87
Artist Name: Os Paralamas Do Sucesso, Total Sold: 45
Artist Name: Deep Purple, Total Sold: 44
Artist Name: Faith No More, Total Sold: 42
Artist Name: Lost, Total Sold: 41
Artist Name: Eric Clapton, Total Sold: 40
Artist Name: R.E.M., Total Sold: 39
